In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import os
import numpy as np
#import ta
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#pip install ta

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime

def save_nse_stocks_data(tickers, output_folder="nse_stock_data"):
    """
    Fetch and save 15-minute stock data for Indian stocks on NSE.
    Each stock's data is saved to a separate CSV file.

    :param tickers: List of stock tickers (e.g., RELIANCE.NS, TCS.NS).
    :param output_folder: Directory to save the CSV files.
    """


    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for ticker in tickers:
        try:
            print(f"Fetching data for {ticker}...")

            # Fetch 15-minute interval data for the last trading day
            data = yf.download(tickers=ticker, period="1d", interval="15m")

            # Check if the index is already timezone-aware
           # print(f"Timezone information: {data.index.tz}")

            # If the index is timezone-naive (no timezone information), localize to UTC
            if data.index.tz is None:
                data.index = data.index.tz_localize("UTC")
            else:
                # If the index already has timezone information, directly convert it to IST
                data.index = data.index.tz_convert("Asia/Kolkata")

            # Display the adjusted datetime (in IST)
            #print(data.index)


            data.reset_index(inplace=True)

           # data.index = data.index.tz_localize("UTC").tz_convert("Asia/Kolkata")
            # Add a column for when the data was fetched
            data['Fetched_At'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')


            # Calculate cumulative price-volume and cumulative volume


            # Define output file path
            output_file = os.path.join(output_folder, f"{ticker.replace('.NS', '')}_15min.csv")

            # Save to CSV (append if file exists)
            if not os.path.exists(output_file):
                data.to_csv(output_file, index=False)
            else:
                data.to_csv(output_file, mode='a', header=False, index=False)

            print(f"Data for {ticker} saved to {output_file}.")

            # from google.colab import files
            # ticker = "TITAN.NS"
            # localfile="C:\\Users\\srudroju\\OneDrive - hsconline\\Sannihith-Reports\\a"
            # local_file = f"{ticker}_15min_data.csv"
            # data.to_csv(local_file)
            # files.download(local_file)

            # uploaded = files.upload()

            # # Get the uploaded file name
            # filename = list(uploaded.keys())[0]
            # uploaded_data = pd.read_csv(filename)

            # Display the first few rows of the uploaded data
            #print(uploaded_data.head())
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

# List of NSE stock tickers
#nse_tickers = ["ZENTEC.NS", "TITAN.NS","RAINBOW.NS","HAL.NS"]
nse_tickers = ["TORNTPOWER.NS"]

# Save the data
save_nse_stocks_data(nse_tickers)



# Download file


[*********************100%***********************]  1 of 1 completed

Fetching data for TORNTPOWER.NS...
Data for TORNTPOWER.NS saved to nse_stock_data/TORNTPOWER_15min.csv.


In [ ]:
# filepath='/content/nse_stock_data/TITAN_15min.csv'
# #filepath='/content/nse_stock_data/TORNTPHARM_15min.csv'
# #filepath='/content/nse_stock_data/RAINBOW_15min.csv'
# ds=pd.read_csv(filepath)
# data=ds.drop(index=0)
# data.head()



,Datetime,Adj Close,Close,High,Low,Open,Volume,Fetched_At
1,2025-01-03 09:15:00,3400.449951171875,3400.449951171875,3412.25,3379.60009765625,3391.75,26793,2025-01-04 09:04:19
2,2025-01-03 09:30:00,3420.60009765625,3420.60009765625,3430.0,3399.85009765625,3400.550048828125,51376,2025-01-04 09:04:19
3,2025-01-03 09:45:00,3444.75,3444.75,3451.89990234375,3418.449951171875,3419.5,69900,2025-01-04 09:04:19
4,2025-01-03 10:00:00,3452.64990234375,3452.64990234375,3458.0,3442.050048828125,3444.75,63231,2025-01-04 09:04:19
5,2025-01-03 10:15:00,3433.0,3433.0,3452.949951171875,3425.39990234375,3452.5,69462,2025-01-04 09:04:19


In [ ]:
filepath='/content/nse_stock_data/ZENTEC_15min.csv'

#filepath='/content/nse_stock_data/TITAN_15min.csv'
#filepath='/content/nse_stock_data/TORNTPHARM_15min.csv'
#filepath='/content/nse_stock_data/RAINBOW_15min.csv'
filepath='/content/nse_stock_data/PANACEABIO_15min.csv'
# filepath= r'C:\Users\srudroju\Downloads\TITAN_NS_15min_data.csv'
# print(filepath)
data=pd.read_csv(filepath)
data=data.drop(index=0)
data.head()
data['High'] = pd.to_numeric(data['High'], errors='coerce')

# Compute the MaxHigh column
data['MaxHigh'] =round(data['High'].max(), 2)
data['Close']= pd.to_numeric(data['Close'], errors='coerce')
data['Open']= pd.to_numeric(data['Open'], errors='coerce')

data['Low']= pd.to_numeric(data['Low'], errors='coerce')


data['MaxLow']=round(data['Low'].min(), 2)
data['TypicalPrice'] = (data['High'] + data['Low'] + data['Close']) / 3
data['TypicalPrice']= pd.to_numeric(data['TypicalPrice'], errors='coerce')
data['Volume']= pd.to_numeric(data['Volume'], errors='coerce')
data['CumPriceVolume'] = (data['TypicalPrice'] * data['Volume']).cumsum()
data['CumVolume'] = data['Volume'].cumsum()
data['VWAP'] = data['CumPriceVolume'] / data['CumVolume']
data['Avg_Volume']=data['Volume'].rolling(window=3).mean()
#data['50-Day SMA'] = data['Close'].rolling(window=7).mean()
data['10-Day EMA']=data['Close'].ewm(span=10, adjust=False).mean()


# Function to implement Moving Average Crossover Strategy for day trading
def intraday_moving_average_crossover(data):
    data["Short_MA"] = data["Close"].rolling(window=5).mean()  # 5-minute MA
    data["Long_MA"] = data["Close"].rolling(window=20).mean()  # 20-minute MA
    data["Signal_MA"] = 0
    data.loc[data["Short_MA"] > data["Long_MA"], "Signal_MA"] = 1
    data.loc[data["Short_MA"] <= data["Long_MA"], "Signal_MA"] = -1
    return data

# Function to implement RSI Strategy for day trading
def intraday_rsi_strategy(data, period=14):
    delta = data["Close"].diff()
    gain = (delta.where(delta > 0, 0)).rolling(period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(period).mean()
    rs = gain / loss
    data["RSI"] = 100 - (100 / (1 + rs))
    data["Signal_RSI"] = 0
    data.loc[data["RSI"] < 30, "Signal_RSI"] = 1  # Oversold
    data.loc[data["RSI"] > 70, "Signal_RSI"] = -1  # Overbought
    return data

# Function to implement Bollinger Bands Strategy for day trading
def intraday_bollinger_bands_strategy(data, period=20):
    data["BB_MA"] = data["Close"].rolling(window=period).mean()
    data["BB_Upper"] = data["BB_MA"] + 2 * data["Close"].rolling(window=period).std()
    data["BB_Lower"] = data["BB_MA"] - 2 * data["Close"].rolling(window=period).std()
    data["Signal_BB"] = 0
    data.loc[data["Close"] < data["BB_Lower"], "Signal_BB"] = 1  # Buy
    data.loc[data["Close"] > data["BB_Upper"], "Signal_BB"] = -1  # Sell
    return data

    #Breatk out strategy
def intraday_breakout_strategy(data, lookback=20):
    # Calculate rolling High and Low
    data["High_Max"] = data["High"].rolling(window=lookback).max()
    data["Low_Min"] = data["Low"].rolling(window=lookback).min()

    # Drop NaN values introduced by rolling
    data.dropna(subset=["High_Max", "Low_Min"], inplace=True)

    # Initialize breakout signal
    data["Signal_Breakout"] = 0

    # Debug: Print the last few rows of the rolling calculations
    print("Debug: Rolling High and Low")
    print(data[["High", "Low", "Close", "High_Max", "Low_Min"]].tail())

    # Apply breakout conditions
    data.loc[data["Close"] > data["High_Max"], "Signal_Breakout"] = 1  # Breakout upward
    data.loc[data["Close"] < data["Low_Min"], "Signal_Breakout"] = -1  # Breakout downward

    # Debug: Check if any signals are generated
    breakout_signals = data[data["Signal_Breakout"] != 0]
    print("Debug: Breakout Signals")
    print(breakout_signals[["Close", "High_Max", "Low_Min", "Signal_Breakout"]])

    if breakout_signals.empty:
        print("No breakout signals were generated. Consider adjusting the lookback period or checking the data.")

    return data


# Function to implement Mean Reversion Strategy for day trading
def intraday_mean_reversion_strategy(data, period=20):
    data["Mean"] = data["Close"].rolling(window=period).mean()
    data["Signal_MR"] = 0
    data.loc[data["Close"] < data["Mean"], "Signal_MR"] = 1  # Below mean
    data.loc[data["Close"] > data["Mean"], "Signal_MR"] = -1  # Above mean
    return data
    #
    #Pivot point
def pivot_points_breakout(data):
    # Calculate pivot points
    data["Pivot"] = (data["High"] + data["Low"] + data["Close"]) / 3
    data["R1"] = 2 * data["Pivot"] - data["Low"]
    data["S1"] = 2 * data["Pivot"] - data["High"]

    # Add breakout signal
    data["Signal_Breakout"] = 0
    data.loc[data["Close"] > data["R1"], "Signal_Breakout"] = 1
    data.loc[data["Close"] < data["S1"], "Signal_Breakout"] = -1

    return data

# Support level adn Restiance level
def calculate_pivot_levels(data):
    # Calculate Pivot Point (P), Resistance (R1, R2), and Support (S1, S2)
    data["Pivot"] = (data["High"] + data["Low"] + data["Close"]) / 3
    data["R1"] = (2 * data["Pivot"]) - data["Low"]
    data["S1"] = (2 * data["Pivot"]) - data["High"]
    data["R2"] = data["Pivot"] + (data["High"] - data["Low"])
    data["S2"] = data["Pivot"] - (data["High"] - data["Low"])

    return data


def calculate_dynamic_support_resistance(data, window=10):
    # Support: Minimum price over the rolling window
    data["Dynamic_Support"] = data["Low"].rolling(window=window).min()
    # Resistance: Maximum price over the rolling window
    data["Dynamic_Resistance"] = data["High"].rolling(window=window).max()
    return data

# Main function to run all strategies for intraday trading
def implement_intraday_strategies(data):

    data=data.copy()
    # Apply strategies
    data = intraday_moving_average_crossover(data)
    data = intraday_rsi_strategy(data)
    data = intraday_bollinger_bands_strategy(data)
    data = intraday_breakout_strategy(data)
    data = intraday_mean_reversion_strategy(data)




    # Plot intraday data with strategies
    plt.figure(figsize=(14, 10))
    plt.plot(data.index, data["Close"], label="Close Price", color="blue")
    plt.plot(data.index, data["Short_MA"], label="Short MA (5m)", color="green", linestyle="--")
    plt.plot(data.index, data["Long_MA"], label="Long MA (20m)", color="red", linestyle="--")
    plt.plot(data.index, data["BB_Upper"], label="Bollinger Upper", color="orange", linestyle=":")
    plt.plot(data.index, data["BB_Lower"], label="Bollinger Lower", color="orange", linestyle=":")
    plt.scatter(data[data["Signal_Breakout"] == 1].index, data[data["Signal_Breakout"] == 1]["Close"], label="Breakout Buy", marker="^", color="green")
    plt.scatter(data[data["Signal_Breakout"] == -1].index, data[data["Signal_Breakout"] == -1]["Close"], label="Breakout Sell", marker="v", color="red")
    plt.title(f"Intraday Trading Strategies for titan")
    plt.xlabel("Time")
    plt.ylabel("Price")
    plt.legend()
    plt.show()

    return data

def calculate_buy_sell_ranges(data, window=10):
    # Dynamic support and resistance
    data["Dynamic_Support"] = data["Low"].rolling(window=window).min()
    data["Dynamic_Resistance"] = data["High"].rolling(window=window).max()

    # Pivot-based levels
    data["Pivot"] = (data["High"] + data["Low"] + data["Close"]) / 3
    data["R1"] = (2 * data["Pivot"]) - data["Low"]
    data["S1"] = (2 * data["Pivot"]) - data["High"]
    data["R2"] = data["Pivot"] + (data["High"] - data["Low"])
    data["S2"] = data["Pivot"] - (data["High"] - data["Low"])

    # Buy and Sell Ranges
    data["Buy_Range"] = data[["Dynamic_Support", "S1", "S2"]].min(axis=1)  # Lowest of all supports
    data["Sell_Range"] = data[["Dynamic_Resistance", "R1", "R2"]].max(axis=1)  # Highest of all resistances

    return data


    # Identify the extreme bullish
def identify_extreme_bullish(data, window=10):
    # Calculate higher highs and higher lows
    data["Higher_Highs"] = data["High"] > data["High"].shift(1)
    data["Higher_Lows"] = data["Low"] > data["Low"].shift(1)
    data["Bullish_Trend"] = data["Higher_Highs"] & data["Higher_Lows"]
    data['CumPriceVolume'] = (data['TypicalPrice'] * data['Volume']).cumsum()
    data['CumVolume'] = data['Volume'].cumsum()
    # Identify periods where price is above VWAP
    data["Above_VWAP"] = data["Close"] > data["VWAP"]

    # Calculate RSI
    delta = data["Close"].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=window).mean()
    avg_loss = pd.Series(loss).rolling(window=window).mean()
    rs = avg_gain / avg_loss
    data["RSI"] = 100 - (100 / (1 + rs))

    # Extreme bullish condition
    data["Extreme_Bullish"] = (data["Bullish_Trend"]) & (data["Above_VWAP"]) & (data["RSI"] > 70)

    return data

def calculate_avg_buy_sell_ranges(data, window=10):
    # Dynamic support and resistance
    data["Dynamic_Support"] = data["Low"].rolling(window=window).min()
    data["Dynamic_Resistance"] = data["High"].rolling(window=window).max()

    # Pivot-based levels
    data["Pivot"] = (data["High"] + data["Low"] + data["Close"]) / 3
    data["R1"] = (2 * data["Pivot"]) - data["Low"]
    data["S1"] = (2 * data["Pivot"]) - data["High"]
    data["R2"] = data["Pivot"] + (data["High"] - data["Low"])
    data["S2"] = data["Pivot"] - (data["High"] - data["Low"])

    # Buy and Sell Ranges
    data["Buy_Range"] = data[["Dynamic_Support", "S1", "S2"]].min(axis=1)  # Lowest of all supports
    data["Sell_Range"] = data[["Dynamic_Resistance", "R1", "R2"]].max(axis=1)  # Highest of all resistances

    # Average Buy and Sell Ranges
    avg_buy_range = data["Buy_Range"].mean()
    avg_sell_range = data["Sell_Range"].mean()

    return avg_buy_range, avg_sell_range


#data1= implement_intraday_strategies(data)

#data=intraday_breakout_strategy(data)
#data.head(10)
#data=calculate_dynamic_support_resistance(data)
#data.tail(10)
#data=identify_extreme_bullish(data)
data=calculate_buy_sell_ranges(data)
avg_buy_range, avg_sell_range = calculate_avg_buy_sell_ranges(data, window=10)
print(avg_buy_range,avg_sell_range)


426.8215525987413 435.54199964735244


In [ ]:
#filepath='/content/nse_stock_data/ZENTEC_15min.csv'

filepath='/content/nse_stock_data/TITAN_15min.csv'
#filepath='/content/nse_stock_data/TORNTPHARM_15min.csv'
#filepath='/content/nse_stock_data/RAINBOW_15min.csv'
ds=pd.read_csv(filepath)
data=ds.drop(index=0)

data['High'] = pd.to_numeric(data['High'], errors='coerce')

# Compute the MaxHigh column
data['MaxHigh'] =round(data['High'].max(), 2)
data['Close']= pd.to_numeric(data['Close'], errors='coerce')
data['Open']= pd.to_numeric(data['Open'], errors='coerce')

data['Low']= pd.to_numeric(data['Low'], errors='coerce')


data['MaxLow']=round(data['Low'].min(), 2)
data['TypicalPrice'] = (data['High'] + data['Low'] + data['Close']) / 3
data['TypicalPrice']= pd.to_numeric(data['TypicalPrice'], errors='coerce')
data['Volume']= pd.to_numeric(data['Volume'], errors='coerce')
data['CumPriceVolume'] = (data['TypicalPrice'] * data['Volume']).cumsum()
data['CumVolume'] = data['Volume'].cumsum()
data['Avg_Volume']=data['Volume'].rolling(window=3).mean()
#data['50-Day SMA'] = data['Close'].rolling(window=7).mean()
data['10-Day EMA']=data['Close'].ewm(span=10, adjust=False).mean()
#print(data.dtypes)
# # Calculate VWAP

# Check conditions for Buy/Sell signals
def signal(row):
    if row['Close'] > row['10-Day EMA']:
        return 'Buy'
    elif row['Close'] < row['10-Day EMA']:
        return 'Sell'
    else:
        return 'Hold'


# Define thresholds for strong signals
distance_threshold = 1.5  # Percentage distance between Close and EMA
volume_threshold = 2000 ;  # High volume threshold

# Signal function
def strong_signal(row):
    distance = ((row['Close'] - row['10-Day EMA']) / row['10-Day EMA']) * 100

    if distance > distance_threshold and row['Volume'] > data['Avg_Volume']:
        return 'Strong Buy'
    elif distance < -distance_threshold and row['Volume'] > data['Avg_Volume']:
        return 'Strong Sell'
    elif row['Close'] > row['10-Day EMA']:
        return 'Buy'
    elif row['Close'] < row['10-Day EMA']:
        return 'Sell'
    else:
        return 'Hold'

# Apply the function to generate signals
data['St_Signal'] = data.apply(strong_signal, axis=1)

def calculate_obv(data):
    data['OBV'] = (data['Volume'] * ((data['Close'].diff() > 0) * 2 - 1)).cumsum()
    return data
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

data['RSI'] = calculate_rsi(data)

data['High-Low'] = data['High'] - data['Low']
data['High-Close'] = abs(data['High'] - data['Close'].shift(1))
data['Low-Close'] = abs(data['Low'] - data['Close'].shift(1))
data['True Range'] = data[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
data['ATR'] = data['True Range'].rolling(window=14).mean()

data['Volume Z-Score'] = (data['Volume'] - data['Volume'].mean()) / data['Volume'].std()
# Apply the function to generate signals
data['Signal'] = data.apply(signal, axis=1)
data['VWAP'] = data['CumPriceVolume'] / data['CumVolume']

def enhanced_signal(row):
    if row['Close'] > row['10-Day EMA'] and row['RSI'] < 30 and row['Volume Z-Score'] > 2:
        return 'Strong Buy'
    elif row['Close'] < row['10-Day EMA'] and row['RSI'] > 70 and row['Volume Z-Score'] > 2:
        return 'Strong Sell'
    elif row['Close'] > row['10-Day EMA']:
        return 'Buy'
    elif row['Close'] < row['10-Day EMA']:
        return 'Sell'
    else:
        return 'Hold'

def calculate_obv(data):
    """
    Calculate the On-Balance Volume (OBV).
    """
    data['OBV'] = 0
    data['OBV'] = data['Volume'] * ((data['Close'] > data['Close'].shift(1)) * 1 +
                                    (data['Close'] < data['Close'].shift(1)) * -1)
    data['OBV'] = data['OBV'].cumsum()
    return data

def generate_signals(data):
    """
    Generate trading signals based on OBV.
    """
    data['OBV_Trend'] = np.where(data['OBV'] > data['OBV'].shift(1), 'Up', 'Down')

    # Define trading signals
    data['Signal'] = 'Hold'  # Default
    data.loc[(data['Close'] > data['Close'].shift(1)) & (data['OBV_Trend'] == 'Up'), 'Signal'] = 'Buy'
    data.loc[(data['Close'] < data['Close'].shift(1)) & (data['OBV_Trend'] == 'Down'), 'Signal'] = 'Sell'
    return data
data = calculate_obv(data)
data = generate_signals(data)
data['Enhance_Signal'] = data.apply(enhanced_signal, axis=1)


# Fetch intraday data


#data['EMA_9'] = ta.trend.ema_indicator(data['Close'], window=9)
data['EMA_9'] = data['Close'].ewm(span=10, adjust=False).mean()
data['EMA_21'] = ta.trend.ema_indicator(data['Close'], window=21)
# data['RSI'] = ta.momentum.rsi(data['Close'], window=14)

# # Generate Buy/Sell Signals
# data['Signal'] = 0
#data.loc[(data['EMA_9'] > data['EMA_21']) & (data['Close'] > data['VWAP']), 'Signal'] = 1  # Buy
data.loc[(data['EMA_9'] > data['EMA_21']) & (data['Close'] > data['VWAP']), 'Signal_VWAP'] = 1 # Buy

data.loc[(data['EMA_9'] < data['EMA_21']) & (data['Close'] < data['VWAP']), 'Signal_VWAP'] =-1  # Sell

# Print signals
# print(data[['Close', 'EMA_9', 'EMA_21', 'RSI', 'VWAP','Signal_VWAP']])
# df=data[['Close', 'EMA_9', 'EMA_21', 'RSI', 'Signal']]
# Calculate MACD and Signal Line
data['EMA_12'] = data['Close'].ewm(span=12, adjust=False).mean()
data['EMA_26'] = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = data['EMA_12'] - data['EMA_26']
data['MASignal'] = data['MACD'].ewm(span=9, adjust=False).mean()

# Calculate Histogram
#data['Histogram'] = data['MACD'] - data['Signal']

# Generate Buy and Sell Signals
data.loc[data['MACD'] > data['MASignal'], 'Signal2'] = 1  # Buy
data.loc[data['MACD'] < data['MASignal'], 'Signal2'] = -1  # Sell

data.drop(columns={'Adj Close','Fetched_At','CumVolume','CumPriceVolume','TypicalPrice'},inplace=True)
data.tail(28)


,Datetime,Close,High,Low,Open,Volume,MaxHigh,MaxLow,Avg_Volume,10-Day EMA,...,OBV_Trend,Enhance_Signal,EMA_9,EMA_21,Signal_VWAP,EMA_12,EMA_26,MACD,MASignal,Signal2
1,2025-01-03 09:15:00,3400.449951,3412.250000,3379.600098,3391.750000,43424,3481.7,3379.6,NaN,3400.449951,...,Down,Hold,3400.449951,NaN,NaN,3400.449951,3400.449951,0.000000,0.000000,NaN
2,2025-01-03 09:30:00,3420.600098,3430.000000,3399.850098,3400.550049,51376,3481.7,3379.6,NaN,3404.113614,...,Up,Buy,3404.113614,NaN,NaN,3403.549974,3401.942555,1.607419,0.321484,1.0
3,2025-01-03 09:45:00,3444.750000,3451.899902,3418.449951,3419.500000,69900,3481.7,3379.6,54900.000000,3411.502048,...,Up,Buy,3411.502048,NaN,NaN,3409.888439,3405.113476,4.774963,1.212180,1.0
4,2025-01-03 10:00:00,3452.649902,3458.000000,3442.050049,3444.750000,63231,3481.7,3379.6,61502.333333,3418.983476,...,Up,Buy,3418.983476,NaN,NaN,3416.467126,3408.634693,7.832433,2.536230,1.0
5,2025-01-03 10:15:00,3433.000000,3452.949951,3425.399902,3452.500000,69462,3481.7,3379.6,67531.000000,3421.531935,...,Down,Buy,3421.531935,NaN,NaN,3419.010645,3410.439531,8.571114,3.743207,1.0
6,2025-01-03 10:30:00,3473.949951,3476.899902,3432.050049,3432.050049,112876,3481.7,3379.6,81856.333333,3431.062483,...,Up,Buy,3431.062483,NaN,NaN,3427.462846,3415.144006,12.318840,5.458334,1.0
7,2025-01-03 10:45:00,3458.899902,3477.100098,3455.300049,3474.000000,75774,3481.7,3379.6,86037.333333,3436.123832,...,Down,Buy,3436.123832,NaN,NaN,3432.299316,3418.385184,13.914132,7.149493,1.0
8,2025-01-03 11:00:00,3469.000000,3470.000000,3457.050049,3458.850098,44511,3481.7,3379.6,77720.333333,3442.101317,...,Up,Buy,3442.101317,NaN,NaN,3437.945575,3422.134429,15.811146,8.881824,1.0
9,2025-01-03 11:15:00,3470.750000,3481.699951,3464.000000,3469.000000,58558,3481.7,3379.6,59614.333333,3447.310169,...,Up,Buy,3447.310169,NaN,NaN,3442.992410,3425.735583,17.256827,10.556824,1.0
10,2025-01-03 11:30:00,3471.600098,3474.899902,3465.500000,3470.750000,37432,3481.7,3379.6,46833.666667,3451.726519,...,Up,Buy,3451.726519,NaN,NaN,3447.393593,3429.132954,18.260638,12.097587,1.0


In [ ]:
output_folder = "/content/nse_stock_data/"
output_file=output_folder + "testfile.csv"
data.to_csv(output_file, mode='a', header=False, index=False)
#data.to_csv(output_file, index=False)

print(f"Dataset saved to {output_file}")

In [ ]:
ds['MaxHigh']=ds['High'].max()
ds.head()

,Datetime,Open,High,Low,Close,Volume,MaxHigh,MinLow,MaxValue,Fetched_At
0,NaN,ZENTEC.NS,ZENTEC.NS,ZENTEC.NS,ZENTEC.NS,ZENTEC.NS,ZENTEC.NS,NaN,NaN,NaN
1,2025-01-01 09:15:00,2455.0,2513.14990234375,2434.0,2492.39990234375,45875,ZENTEC.NS,NaN,NaN,2025-01-01 12:48:57
2,2025-01-01 09:30:00,2488.300048828125,2497.800048828125,2450.050048828125,2455.0,22903,ZENTEC.NS,NaN,NaN,2025-01-01 12:48:57
3,2025-01-01 09:45:00,2461.10009765625,2485.0,2451.14990234375,2470.0,15828,ZENTEC.NS,NaN,NaN,2025-01-01 12:48:57
4,2025-01-01 10:00:00,2472.550048828125,2493.89990234375,2465.0,2480.0,9377,ZENTEC.NS,NaN,NaN,2025-01-01 12:48:57


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime

def fetch_last_10_days_data(tickers, output_folder="nse_stock_data"):
    """
    Fetch and save the last 10 days of 15-minute stock data for multiple NSE stocks.
    Ensures the data is filtered to NSE trading hours (9:15 AM to 3:30 PM IST).

    :param tickers: List of stock tickers (e.g., RELIANCE.NS, TCS.NS).
    :param output_folder: Directory to save the CSV files.
    """
    import os

    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for ticker in tickers:
        try:
            print(f"Fetching 15-min data for the last 10 days: {ticker}...")

            # Fetch 15-minute interval data for the last 10 days
            data = yf.download(tickers=ticker, period="5d", interval="15m")

            # Handle case where no data is returned
            if data.empty:
                print(f"No data available for {ticker}. Skipping...")
                continue

            data.reset_index(inplace=True)

            # Convert UTC to IST
            data['Datetime'] = data['Datetime'].dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
            # Remove time zone info (make naive datetime)
            data['Datetime'] = data['Datetime'].dt.tz_localize(None)
            # Set 'Datetime' as the index
            data.set_index('Datetime', inplace=True)

            # Filter data for NSE trading hours (9:15 AM to 3:30 PM)
            data = data.between_time("09:15", "15:30")

            # Add a column for when the data was fetched
            data['Fetched_At'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            # Reset index after filtering
            data.reset_index(inplace=True)

            # Select and reorder the columns
            data = data[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Fetched_At']]

            # Define output file path
            output_file = os.path.join(output_folder, f"{ticker.replace('.NS', '')}_15min_last10days.csv")

            # Save to CSV
            data.to_csv(output_file, index=False)
            print(f"Data for {ticker} saved to {output_file}.")

        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

# List of NSE stock tickers
nse_tickers = ["ZENTEC.NS", "TITAN.NS"]

# Fetch and save the data
fetch_last_10_days_data(nse_tickers)


[*********************100%***********************]  1 of 1 completed

Fetching 15-min data for the last 10 days: ZENTEC.NS...
Data for ZENTEC.NS saved to nse_stock_data/ZENTEC_15min_last10days.csv.
Fetching 15-min data for the last 10 days: TITAN.NS...



[*********************100%***********************]  1 of 1 completed

Data for TITAN.NS saved to nse_stock_data/TITAN_15min_last10days.csv.


In [ ]:
filepath10='/content/nse_stock_data/ZENTEC_15min_last10days.csv'
ds10=pd.read_csv(filepath10)
ds10.tail(5)

,Datetime,Open,High,Low,Close,Volume,Fetched_At
16,2024-12-31 15:30:00,2371.449951171875,2373.0,2345.0,2355.14990234375,26361,2025-01-01 11:44:16
17,2025-01-01 14:45:00,2455.0,2513.14990234375,2434.0,2492.39990234375,37981,2025-01-01 11:44:16
18,2025-01-01 15:00:00,2488.300048828125,2497.800048828125,2450.050048828125,2455.0,22903,2025-01-01 11:44:16
19,2025-01-01 15:15:00,2461.10009765625,2485.0,2451.14990234375,2470.0,15828,2025-01-01 11:44:16
20,2025-01-01 15:30:00,2472.550048828125,2493.89990234375,2465.0,2480.0,9377,2025-01-01 11:44:16
